## Read google sheet into Pandas dataframe

https://developers.google.com/sheets/api/reference/rest/v4/spreadsheets#Spreadsheet
https://developers.google.com/identity/protocols/OAuth2ServiceAccount
https://console.developers.google.com/apis/api/sheets.googleapis.com/overview?project=flypacifica&folder&organizationId&duration=PT1H

In [1]:
# %%capture
# Read private constants like API key and sheet URL
# %run secrets.ipynb

In [2]:
def get_service():
    scopes = ["https://www.googleapis.com/auth/spreadsheets"]
    # Prepare credentials
    credentials = service_account.Credentials.from_service_account_file(
        GOOGLE_SHEETS_KEY,
        scopes=scopes,
    )
    # Create service
    service = googleapiclient.discovery.build(
        "sheets",
        "v4",
        credentials=credentials,
    )
    return service


def read_sheet(sheet_id, col_range):
    service = get_service()
    request = service.spreadsheets().values().get(
        spreadsheetId=sheet_id,
        range=col_range,
    )
    response = request.execute()
    cols = response["values"][0]
    rows = response["values"][1:]
    df = pd.DataFrame(columns=cols)
    
    row_dfs = []
    for row in rows:
        row_dict = dict(zip(cols, row))
        row_dfs.append(pd.DataFrame(row_dict, index=[0]))

    return pd.concat(row_dfs)